In [1]:
import pickle
import os
os.environ['TRANSFORMERS_CACHE'] = "../models"
from sentence_transformers import SentenceTransformer, util

c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# pkl 파일 로드
file_path = "../data/final_result.pkl"  # pkl 파일 경로를 지정하세요.

with open(file_path, "rb") as f:
    faq_data = pickle.load(f)


In [10]:
# model_name = "all-MiniLM-L6-v2"
# model_name = "dragonkue/BGE-m3-ko"
# model_name = "jhgan/ko-sroberta-multitask"
model_name = "upskyy/bge-m3-korean"
# model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# model_name = "nlpai-lab/KoE5"

In [11]:
# FAQ 데이터 질문 벡터화
# - faq_questions: FAQ 데이터의 질문 리스트
# - faq_embeddings: FAQ 질문들을 벡터로 변환 (한 번만 수행됨)

model = SentenceTransformer(model_name)
faq_questions = list(faq_data.keys())
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)

In [5]:
file_path = "../data/test_question.txt"

with open(file_path, "r", encoding="utf-8") as file:
    questions = [text.strip() for text in file.readlines()]
questions

['상품 등록이 안되요',
 '결제수단을 변경하고 싶어요',
 '할인쿠폰 발급하려면 어떻게 해야되나요?',
 '고객리뷰를 삭제하고 싶어요',
 '배송조회는 어떻게 하나요?',
 '판매수수료는 어떻게 볼 수 있나요?',
 '계정을 탈퇴하고 싶어요',
 '오늘 날씨는 어때요?',
 '여의도 근처 맛집 추천 좀 해주세요.',
 'BTS 멤버는 몇 명인가요?']

In [6]:
# # 특정 단어 검색 함수
# def search_in_dict(data, keyword):
#     """
#     주어진 딕셔너리에서 특정 키워드가 포함된 key와 value를 찾는 함수.
#     :param data: 딕셔너리 (예: faq_data)
#     :param keyword: 검색할 키워드 (예: "스마트스토어")
#     :return: 검색 결과 리스트
#     """
#     results = []
#     for key, value in data.items():
#         if keyword in key:  # key에서 키워드 포함 여부 확인
#             results.append((key, value))
#     return results

# # 키워드 검색
# keyword = "결제수단"
# matches = search_in_dict(faq_data, keyword)
# matches

In [9]:
for question in questions:
    # 사용자 질문을 벡터로 변환
    query_embedding = model.encode(question, convert_to_tensor=True)
    
    # FAQ 데이터와 코사인 유사도 계산
    scores = util.pytorch_cos_sim(query_embedding, faq_embeddings)
    
    # 가장 높은 유사도를 가진 FAQ 질문의 인덱스 및 점수 반환
    best_idx = scores.argmax().item()
    print("사용자 질문: %s"%question)
    print("유사 질문: %s"%faq_questions[best_idx])
    print("유사도: %s"%scores[0][best_idx].item())
    print("-"*100)

사용자 질문: 상품 등록이 안되요
유사 질문: ★ 상품등록하는 방법
유사도: 0.7856465578079224
----------------------------------------------------------------------------------------------------
사용자 질문: 결제수단을 변경하고 싶어요
유사 질문: 취소 환불 시 구매자가 환불받을 계좌를 변경할 수 있나요?
유사도: 0.5694704055786133
----------------------------------------------------------------------------------------------------
사용자 질문: 할인쿠폰 발급하려면 어떻게 해야되나요?
유사 질문: [혜택 등록] 쿠폰 발행 할때 최대 할인은 얼마까지 되나요?
유사도: 0.6560598015785217
----------------------------------------------------------------------------------------------------
사용자 질문: 고객리뷰를 삭제하고 싶어요
유사 질문: 베스트리뷰(스토어Pick)로 선정한 리뷰는 고객이 삭제할 수 없나요?
유사도: 0.7427887320518494
----------------------------------------------------------------------------------------------------
사용자 질문: 배송조회는 어떻게 하나요?
유사 질문: 희망일배송 주문 확인은 어떻게 하나요?
유사도: 0.7133914232254028
----------------------------------------------------------------------------------------------------
사용자 질문: 판매수수료는 어떻게 볼 수 있나요?
유사 질문: 수수료는 어떤 금액 기준으로 적용 되나요?
유사도: 0.6526105999946594